# Parse Hyperparameters from Tune

This notebook parses hyperparameter and validation statistics from the tune runs

In [1]:
import os
import pandas as pd

!rsync -avz -e "ssh" marc@uni:/data2/breizhcrops/tune /tmp/
    
tuneresults = "/tmp/tune"
runs = os.listdir(tuneresults)
runs = [run for run in runs if os.path.exists(os.path.join(tuneresults,run,"trainlog.csv"))]

receiving incremental file list
tune/
tune/InceptionTime_input-dim=13_num-classes=9_hidden-dims=128_num-layers=3_learning-rate=0.0009980594748806762_weight-decay=0.0010626996351948946/trainlog.csv
tune/InceptionTime_input-dim=13_num-classes=9_hidden-dims=64_num-layers=4_learning-rate=0.0005930998594456241_weight-decay=1.8660112778851542e-05/trainlog.csv
tune/LSTM_input-dim=13_num-classes=9_hidden-dims=128_num-layers=1_bidirectional=False_use-layernorm=True_dropout=0.7369083498541_learning-rate=0.00013807584269740695_weight-decay=0.000944157330485858/trainlog.csv
tune/StarRNN_input-dim=13_num-classes=9_hidden-dims=32_num-layers=1_learning-rate=0.00023598471456262645_weight-decay=0.0026847776440634013/
tune/StarRNN_input-dim=13_num-classes=9_hidden-dims=32_num-layers=1_learning-rate=0.00023598471456262645_weight-decay=0.0026847776440634013/trainlog.csv
tune/StarRNN_input-dim=13_num-classes=9_hidden-dims=32_num-layers=3_learning-rate=0.00020615099183257646_weight-decay=2.3806109614980892e

In [2]:
def parse_hyperparameters_folder_name(run):
    
    run = os.path.basename(run)
    model, *hyperparameter_str = run.split("_")

    hyperparameter = dict()
    for kv in hyperparameter_str:
        k,v = kv.split("=")
        hyperparameter[k] = v
    hyperparameter["model"] = model
    return hyperparameter

In [3]:
def read_best_epoch_stats(run):
    df = pd.read_csv(os.path.join(run,"trainlog.csv"), index_col=0)
    bestepoch = df["testloss"].idxmin()
    best_stats = df.loc[bestepoch].to_dict()
    best_stats["epoch"] = bestepoch
    return best_stats

In [4]:

def merge(dict1,dict2):
    for k,v in dict2.items():
        dict1[k] = v
    return dict1

In [17]:
results = list()
for run in runs:
    result = parse_hyperparameters_folder_name(os.path.join(tuneresults,run))
    best_epoch_stats = read_best_epoch_stats(os.path.join(tuneresults,run))
    result = merge(result,best_epoch_stats)
    results.append(result)
results = pd.DataFrame(results)

models = results.model.unique()
tables = dict()
for model in models:
    df = results.loc[results.model == model]
    tables[model] = df.sort_values(by="testloss")

In [ ]:
tables["LSTM"].dropna(axis=1)

In [ ]:
tables["TransformerEncoder"].dropna(axis=1)

In [ ]:
tables["MSResNet"].dropna(axis=1)

In [ ]:
tables["tempcnn"].dropna(axis=1)

In [ ]:
tables["StarRNN"].dropna(axis=1)

In [ ]:
tables["InceptionTime"].dropna(axis=1)

In [20]:
tables["InceptionTime"].dropna(axis=1)

,input-dim,num-classes,hidden-dims,num-layers,bidirectional,use-layernorm,dropout,learning-rate,weight-decay,model,...,kappa,precision_macro,precision_micro,precision_weighted,recall_macro,recall_micro,recall_weighted,testloss,trainloss,epoch
8,13,9,64,1,False,True,0.6258258913324922,0.0025221824203116097,1.4395817709449859e-08,LSTM,...,0.665910,0.514199,0.748418,0.741844,0.518073,0.748418,0.748418,0.420265,0.477915,4
13,13,9,32,2,False,True,0.43241110353282225,0.0020093743995764023,1.8914669258114296e-06,LSTM,...,0.577795,0.408765,0.677531,0.684683,0.456824,0.677531,0.677531,0.606063,0.725590,1
23,13,9,128,4,False,True,0.1520615328684862,0.0001980167410450375,0.0003525962044101824,LSTM,...,0.571468,0.390827,0.670715,0.676560,0.443769,0.670715,0.670715,0.621401,0.736335,3
14,13,9,64,2,False,True,0.6244136602431383,0.00016299539236650087,1.5141889012034385e-08,LSTM,...,0.567790,0.395593,0.668035,0.668521,0.430963,0.668035,0.668035,0.650203,0.737174,4
0,13,9,128,4,False,True,0.6178803948247342,0.00015559551450284578,4.9669739192664105e-06,LSTM,...,0.553699,0.376317,0.653395,0.664700,0.437673,0.653395,0.653395,0.663972,0.744139,2
19,13,9,64,2,False,True,0.10117426826459078,0.0003497281923436781,3.15905367632669e-07,LSTM,...,0.537815,0.398916,0.648803,0.669874,0.417390,0.648803,0.648803,0.714445,0.902205,1
28,13,9,32,2,False,True,0.2578036739800457,0.00014242006722663868,1.9360156412014993e-08,LSTM,...,0.539148,0.358364,0.652006,0.638978,0.340686,0.652006,0.652006,0.746085,0.790685,4
1,13,9,128,1,False,True,0.7369083498541,0.00013807584269740695,0.000944157330485858,LSTM,...,0.552410,0.372567,0.658155,0.648329,0.378454,0.658155,0.658155,0.750018,0.768852,3


In [21]:
tables["TransformerEncoder"].dropna(axis=1)

,input-dim,num-classes,dropout,learning-rate,weight-decay,model,accuracy,f1_macro,f1_micro,f1_weighted,...,recall_macro,recall_micro,recall_weighted,testloss,trainloss,epoch,d-model,d-inner,n-layers,n-head
2,13,9,0.37526788159446334,0.00015000702046192362,0.006893919659116325,TransformerEncoder,0.732942,0.506822,0.732942,0.731597,...,0.506735,0.732942,0.732942,0.475739,0.579781,3,512,2048,2,1
7,13,9,0.4351899794783582,0.0004799366269423782,0.003199092810296778,TransformerEncoder,0.735977,0.499088,0.735977,0.732709,...,0.508277,0.735977,0.735977,0.501770,0.558779,4,32,128,4,1


In [22]:
tables["MSResNet"].dropna(axis=1)

,input-dim,num-classes,hidden-dims,learning-rate,weight-decay,model,accuracy,f1_macro,f1_micro,f1_weighted,kappa,precision_macro,precision_micro,precision_weighted,recall_macro,recall_micro,recall_weighted,testloss,trainloss,epoch
26,13,9,32,0.0002773797832973116,0.00015515037808987102,MSResNet,0.767932,0.535158,0.767932,0.763729,0.696542,0.613397,0.767932,0.765151,0.531539,0.767932,0.767932,0.400126,0.413298,3
9,13,9,32,0.0001319164345391069,0.00033765428641244135,MSResNet,0.751573,0.503003,0.751573,0.728347,0.671282,0.511788,0.751573,0.744929,0.517723,0.751573,0.751573,0.443173,0.537561,1
18,13,9,64,0.00010844961356253675,1.0860785318494814e-05,MSResNet,0.758737,0.507638,0.758737,0.736770,0.683028,0.512502,0.758737,0.746622,0.527581,0.758737,0.758737,0.450648,0.432449,2
15,13,9,32,0.00010099699324166293,5.221576864451801e-05,MSResNet,0.762175,0.514683,0.762175,0.738031,0.685809,0.518795,0.762175,0.748505,0.531435,0.762175,0.762175,0.458663,0.481721,4
16,13,9,32,0.0006271686393146093,4.750234747127917e-06,MSResNet,0.747643,0.493787,0.747643,0.739113,0.662943,0.550471,0.747643,0.756383,0.464259,0.747643,0.747643,0.488207,0.473864,1
3,13,9,32,0.00010586983718434764,1.4729300901739634e-07,MSResNet,0.756363,0.510066,0.756363,0.734524,0.675558,0.532321,0.756363,0.751641,0.507440,0.756363,0.756363,0.498464,0.605760,1


In [24]:
tables["tempcnn"].dropna(axis=1)

KeyError: 'tempcnn'

In [27]:
tables["StarRNN"].dropna(axis=1)

,input-dim,num-classes,hidden-dims,num-layers,learning-rate,weight-decay,model,accuracy,f1_macro,f1_micro,...,kappa,precision_macro,precision_micro,precision_weighted,recall_macro,recall_micro,recall_weighted,testloss,trainloss,epoch
20,13,9,64,2,0.0023827838758872675,4.686585523423313e-08,StarRNN,0.746699,0.496543,0.746699,...,0.671124,0.497666,0.746699,0.743545,0.518404,0.746699,0.746699,0.448618,0.549433,2
5,13,9,32,1,0.0017087114553940837,8.562736523247137e-07,StarRNN,0.718879,0.463624,0.718879,...,0.635236,0.447596,0.718879,0.714092,0.502437,0.718879,0.718879,0.466813,0.700335,3
12,13,9,32,4,0.0031060551507801166,0.003186307296525995,StarRNN,0.748923,0.526275,0.748923,...,0.670997,0.528503,0.748923,0.751510,0.526388,0.748923,0.748923,0.473416,0.487888,1
6,13,9,32,2,0.0005010089666089117,0.00071707897516736,StarRNN,0.714203,0.449563,0.714203,...,0.626222,0.448372,0.714203,0.696309,0.458137,0.714203,0.714203,0.539079,0.705777,4
22,13,9,32,3,0.003762732217262833,1.8233116840509476e-08,StarRNN,0.652577,0.316578,0.652577,...,0.545482,0.415495,0.652577,0.656350,0.325576,0.652577,0.652577,0.769885,0.862189,1
4,13,9,32,4,0.0004053566793428001,1.2965553875848835e-06,StarRNN,0.622239,0.244242,0.622239,...,0.490292,0.261748,0.622239,0.541661,0.285494,0.622239,0.622239,0.818904,1.022651,3
24,13,9,32,3,0.00020615099183257646,2.3806109614980892e-06,StarRNN,0.479329,0.193565,0.479329,...,0.294233,0.224329,0.479329,0.449756,0.208746,0.479329,0.479329,1.257567,2.296237,0


In [28]:
tables["InceptionTime"].dropna(axis=1)

,input-dim,num-classes,hidden-dims,num-layers,learning-rate,weight-decay,model,accuracy,f1_macro,f1_micro,...,kappa,precision_macro,precision_micro,precision_weighted,recall_macro,recall_micro,recall_weighted,testloss,trainloss,epoch
17,13,9,64,4,0.00028098649833653664,0.0008642986871646551,InceptionTime,0.769675,0.530594,0.769675,...,0.696469,0.537773,0.769675,0.761159,0.530990,0.769675,0.769675,0.427361,0.430931,4
25,13,9,64,3,0.004050799676190329,3.4823242124186607e-06,InceptionTime,0.718242,0.428212,0.718242,...,0.625564,0.503016,0.718242,0.740064,0.442941,0.718242,0.718242,0.552263,0.493350,1
10,13,9,128,4,0.0004913331553599582,0.0004194548892874661,InceptionTime,0.689358,0.467398,0.689358,...,0.603722,0.489287,0.689358,0.748213,0.490627,0.689358,0.689358,0.558810,2.526230,0
21,13,9,32,3,0.00030278311415271836,1.0901342315664123e-05,InceptionTime,0.725135,0.469096,0.725135,...,0.628245,0.535087,0.725135,0.755333,0.484816,0.725135,0.725135,0.595591,0.475577,4
27,13,9,128,3,0.0009980594748806762,0.0010626996351948946,InceptionTime,0.743983,0.471847,0.743983,...,0.664273,0.469169,0.743983,0.742527,0.518542,0.743983,0.743983,0.598770,0.504577,1
11,13,9,64,4,0.0005930998594456241,1.8660112778851542e-05,InceptionTime,0.614643,0.349389,0.614643,...,0.520370,0.361941,0.614643,0.658225,0.470646,0.614643,0.614643,0.867144,0.500636,1
